In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=2e166a8356cb9d90dcbe7f66ad6a8fe905de076340206449095964efda9283d4
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
!wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz
!wget https://kdd.ics.uci.edu/databases/kddcup99/kddcup.names

--2024-04-11 12:02:50--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18115902 (17M) [application/x-gzip]
Saving to: ‘kddcup.data.gz’

kddcup.data.gz      100%[===================>]  17.28M  5.51MB/s    in 3.1s    

2024-04-11 12:02:53 (5.51 MB/s) - ‘kddcup.data.gz’ saved [18115902/18115902]



In [ ]:
!gzip -d kddcup.data.gz

In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('KMeans').getOrCreate()

In [ ]:
df = spark.read.csv('kddcup.data', header='False', inferSchema='True')

df.show(5)

+---+---+----+---+---+-----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|_c0|_c1| _c2|_c3|_c4|  _c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|   _c41|
+---+---+----+---+---+-----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|  0|tcp|http| SF|215|45076|  0|  0|  0|  0|   0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   1| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0|   0|   0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|normal.|
|  0|tcp|http| SF|162| 4528|  0|  0|  0|  0|   0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   2|   2| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0|   1|   1| 1.0| 0.0

In [ ]:
print('Number of datapoints: ', df.count())

Number of datapoints:  4898431


In [ ]:
columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot','num_failed_logins', 'logged_in',
    'num_compromised', 'root_shell', 'su_attempted', 'num_root',
    'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds',
    'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
    'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate', 'label'
]

In [ ]:
df = df.toDF(*columns)

df.show(5)

+--------+-------------+-------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_hos

In [ ]:
from pyspark.sql.functions import col

In [ ]:
df.select('label').groupby('label').count().orderBy(col('count'), ascending=False).show(10)

+------------+-------+
|       label|  count|
+------------+-------+
|      smurf.|2807886|
|    neptune.|1072017|
|     normal.| 972781|
|      satan.|  15892|
|    ipsweep.|  12481|
|  portsweep.|  10413|
|       nmap.|   2316|
|       back.|   2203|
|warezclient.|   1020|
|   teardrop.|    979|
+------------+-------+
only showing top 10 rows



In [ ]:
df.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- num_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true

In [ ]:
from pyspark.sql.types import StringType

all_cols = df.schema.fields

string_cols = [col.name for col in all_cols if col.dataType == StringType()]

print(string_cols)

['protocol_type', 'service', 'flag', 'label']


In [ ]:
string_cols.remove('label')

print(string_cols)

['protocol_type', 'service', 'flag']


In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

indexers = [
    StringIndexer(inputCol=column, outputCol=column+'_indexed')
    for column in string_cols
]

pipeline = Pipeline(stages=indexers)
pipeline_model = pipeline.fit(df)

In [ ]:
df = pipeline_model.transform(df)

df.show(5)

+--------+-------------+-------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+---------------------+---------------+------------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|sam

In [ ]:
from pyspark.ml.feature import OneHotEncoder

encoders = [
    OneHotEncoder(inputCol=column+'_indexed', outputCol=column+'_vec')
    for column in string_cols
]

pipeline = Pipeline(stages=encoders)
pipeline_model = pipeline.fit(df)

In [ ]:
df = pipeline_model.transform(df)

df.show(5)

+--------+-------------+-------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+---------------------+---------------+------------+-----------------+--------------+--------------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate

In [ ]:
columns = [column if column not in string_cols else column+'_vec' for column in columns]

print(columns)

['duration', 'protocol_type_vec', 'service_vec', 'flag_vec', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label']


In [ ]:
df = df.select(*columns)

df.show(5)

+--------+-----------------+--------------+--------------+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type_vec|   service_vec|      flag_vec|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rat

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler_cols = columns[:-1]

assembler = VectorAssembler(inputCols=assembler_cols, outputCol='featureVector')

In [ ]:
df = assembler.transform(df)

df.show(5)

+--------+-----------------+--------------+--------------+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+--------------------+
|duration|protocol_type_vec|   service_vec|      flag_vec|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rer

In [ ]:
columns = ['featureVector', 'label']

df = df.select(*columns)

df.show(5)

+--------------------+-------+
|       featureVector|  label|
+--------------------+-------+
|(119,[2,5,72,82,8...|normal.|
|(119,[2,5,72,82,8...|normal.|
|(119,[2,5,72,82,8...|normal.|
|(119,[2,5,72,82,8...|normal.|
|(119,[2,5,72,82,8...|normal.|
+--------------------+-------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(
    inputCol='featureVector', outputCol='scaledFeatures',
    withMean=False, withStd=True
)

scaler_model = scaler.fit(df)

In [ ]:
df = scaler_model.transform(df)

df.show(5)

+--------------------+-------+--------------------+
|       featureVector|  label|      scaledFeatures|
+--------------------+-------+--------------------+
|(119,[2,5,72,82,8...|normal.|(119,[2,5,72,82,8...|
|(119,[2,5,72,82,8...|normal.|(119,[2,5,72,82,8...|
|(119,[2,5,72,82,8...|normal.|(119,[2,5,72,82,8...|
|(119,[2,5,72,82,8...|normal.|(119,[2,5,72,82,8...|
|(119,[2,5,72,82,8...|normal.|(119,[2,5,72,82,8...|
+--------------------+-------+--------------------+
only showing top 5 rows



In [ ]:
columns = ['scaledFeatures', 'label']

df = df.select(*columns)

df.show(5)

+--------------------+-------+
|      scaledFeatures|  label|
+--------------------+-------+
|(119,[2,5,72,82,8...|normal.|
|(119,[2,5,72,82,8...|normal.|
|(119,[2,5,72,82,8...|normal.|
|(119,[2,5,72,82,8...|normal.|
|(119,[2,5,72,82,8...|normal.|
+--------------------+-------+
only showing top 5 rows



In [ ]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=100, featuresCol='scaledFeatures', predictionCol='cluster', maxIter=40, tol=1.0e-5)
kmeans_model = kmeans.fit(df)

In [ ]:
print('Training Cost: ', kmeans_model.summary.trainingCost)

Training Cost:  50873186.2900652


In [ ]:
df = kmeans_model.transform(df)

df.show(5)

+--------------------+-------+-------+
|      scaledFeatures|  label|cluster|
+--------------------+-------+-------+
|(119,[2,5,72,82,8...|normal.|     13|
|(119,[2,5,72,82,8...|normal.|     13|
|(119,[2,5,72,82,8...|normal.|     13|
|(119,[2,5,72,82,8...|normal.|     13|
|(119,[2,5,72,82,8...|normal.|     13|
+--------------------+-------+-------+
only showing top 5 rows



In [ ]:
df.select('cluster', 'label').groupBy('cluster', 'label').count().orderBy('cluster', 'label').show()

+-------+----------+-------+
|cluster|     label|  count|
+-------+----------+-------+
|      0|  neptune.| 365281|
|      0|portsweep.|    143|
|      1|  ipsweep.|     40|
|      1|     nmap.|      6|
|      1|   normal.|   3495|
|      1|      pod.|     82|
|      1|portsweep.|      2|
|      1|    satan.|      7|
|      1|    smurf.|2807852|
|      2|  ipsweep.|     13|
|      2|  neptune.|   1046|
|      2|   normal.|     38|
|      2|portsweep.|     11|
|      2|    satan.|      3|
|      3|  neptune.|   1041|
|      3|portsweep.|      9|
|      3|    satan.|      2|
|      4|  ipsweep.|     13|
|      4|  neptune.|   1041|
|      4|portsweep.|     13|
+-------+----------+-------+
only showing top 20 rows

